In [9]:
!pip install streamlit
!pip install gradio
!pip install faker

  Obtaining dependency information for faker from https://files.pythonhosted.org/packages/73/51/cbc859707aa0fc0ad3819ffb3bdaeee28d10d5ef30150ed9d16691ac3795/Faker-19.6.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.7 MB 1.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.7 MB 2.6 MB/s eta 0:00:01
   --------------- ------------------------ 0.7/1.7 MB 4.8 MB/s eta 0:00:01
   ------------------------- -------------- 1.1/1.7 MB 5.8 MB/s eta 0:00:01
   ----------------------------------- ---- 1.5/1.7 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.5 MB/s eta 0:00:00


In [3]:
!pip install spacy

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/34/1a/cd4f39dcd28628bf69d2a6642aea3854f2de877196c737bf08e4ef3ca372/spacy-3.6.1-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/ed/9d/d62d12e3ecc6f99eddea6289413669a905d2ebb15cf9fe075336ca6cceaa/murmurhash-1.0.10-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for cymem<2.1.0,>=2.0.2 from https://files.pythonhosted.org/packages/51/12/4aa9eec680c6d12b2275d479e159c3d063d7c757175063dd45386e15b39d/cymem-2.0.8-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for preshed<3.1.0,>=3.0.2 from https://files.pythonhosted.org/packages/f3/72/108426ca3b6

In [13]:
!spacy download en_core_web_md

     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     --------------------------------------- 0.0/42.8 MB 435.7 kB/s eta 0:01:39
     --------------------------------------- 0.1/42.8 MB 563.7 kB/s eta 0:01:16
     --------------------------------------- 0.1/42.8 MB 944.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/42.8 MB 944.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/42.8 MB 944.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/42.8 MB 944.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/42.8 MB 944.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/42.8 MB 944.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/42.8 MB 944.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/42.8 MB 944.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/42.8 MB

In [11]:
%%writefile app.py

import spacy
import streamlit as st
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the spaCy language model
nlp = spacy.load("en_core_web_md")

# Function to extract named entities (skills, experience, education, projects)
def extract_entities(text):
    lines = text.split('\n')
    skills = []
    experience = []
    education = []
    projects = []

    for line in lines:
        parts = line.strip().split(':')
        if len(parts) == 2:
            entity_type, entity_text = parts[0].strip(), parts[1].strip()
            if entity_type == "SKILL":
                skills.append(entity_text)
            elif entity_type == "EXPERIENCE":
                experience.append(entity_text)
            elif entity_type == "EDUCATION":
                education.append(entity_text)
            elif entity_type == "PROJECT":
                projects.append(entity_text)

    return skills, experience, education, projects

# Streamlit UI
st.title("Resume Ranking App")

# User input for job description
job_description = st.text_area("Enter the job description:")

# User input for resumes
st.write("Enter the resumes in entity format. Use the following format:")
st.write("ENTITY_TYPE: ENTITY_TEXT")
st.write("Example:")
st.write("SKILL: Python, machine learning\nEXPERIENCE: Worked on various projects, including a recommendation system.\n")
resumes_text = st.text_area("Enter the resumes (one per line):")

if job_description and resumes_text:
    # Combine the resumes and the job description into one list
    resumes = resumes_text.strip().split('\n')
    all_text = resumes + [job_description]

    # Create a CountVectorizer to convert text to a matrix of token counts
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(all_text)

    # Calculate cosine similarity between the job description and all resumes
    cosine_similarities = cosine_similarity(X[-1], X[:-1])

    # Create a list of (resume, similarity score) pairs
    resume_scores = list(zip(resumes, cosine_similarities[0]))

    # Sort the resumes by similarity score in descending order
    resume_scores.sort(key=lambda x: x[1], reverse=True)

    # Display the ranked resumes with named entities
    st.header("Ranked Resumes:")
    for i, (resume, score) in enumerate(resume_scores):
        skills, experience, education, projects = extract_entities(resume)
        st.subheader(f"Rank {i + 1}: Similarity Score = {score:.4f}")
        st.write("Skills:", ", ".join(skills))
        st.write("Experience:", ", ".join(experience))
        st.write("Education:", ", ".join(education))
        st.write("Projects:", ", ".join(projects))
        st.write("Resume:", resume)
        st.write("---")


Writing app.py


## Version 2

In [ ]:
import spacy
import streamlit as st
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the spaCy language model
nlp = spacy.load("en_core_web_md")

# Function to extract named entities (skills, experience, education, projects, and candidate name)
def extract_entities(text):
    lines = text.split('\n')
    skills = []
    experience = []
    education = []
    projects = []
    candidate_name = None

    for line in lines:
        parts = line.strip().split(':')
        if len(parts) == 2:
            entity_type, entity_text = parts[0].strip(), parts[1].strip()
            if entity_type == "SKILL":
                skills.append(entity_text)
            elif entity_type == "EXPERIENCE":
                experience.append(entity_text)
            elif entity_type == "EDUCATION":
                education.append(entity_text)
            elif entity_type == "PROJECT":
                projects.append(entity_text)
            elif entity_type == "CANDIDATE_NAME":
                candidate_name = entity_text

    return skills, experience, education, projects, candidate_name

# Streamlit UI
st.title("Resume Ranking App")

# User input for job description
job_description = st.text_area("Enter the job description:")

# User input for resumes
st.write("Enter the resumes in entity format. Use the following format:")
st.write("ENTITY_TYPE: ENTITY_TEXT")
st.write("Example:")
st.write("SKILL: Python, machine learning\nEXPERIENCE: Developed machine learning models for sentiment analysis.\nCANDIDATE_NAME: John Doe\n")
resumes_text = st.text_area("Enter the resumes (one per line):")

if job_description and resumes_text:
    # Combine the resumes and the job description into one list
    resumes = resumes_text.strip().split('\n')
    all_text = resumes + [job_description]

    # Create a CountVectorizer to convert text to a matrix of token counts
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(all_text)

    # Calculate cosine similarity between the job description and all resumes
    cosine_similarities = cosine_similarity(X[-1], X[:-1])

    # Create a list of (resume, similarity score) pairs
    resume_scores = list(zip(resumes, cosine_similarities[0]))

    # Sort the resumes by similarity score in descending order
    resume_scores.sort(key=lambda x: x[1], reverse=True)

    # Display the ranked resumes with named entities, including candidate names
    st.header("Ranked Resumes:")
    for i, (resume, score) in enumerate(resume_scores):
        skills, experience, education, projects, candidate_name = extract_entities(resume)
        st.subheader(f"Rank {i + 1}: Similarity Score = {score:.4f}")
        st.write("Candidate Name:", candidate_name)
        st.write("Skills:", ", ".join(skills))
        st.write("Experience:", ", ".join(experience))
        st.write("Education:", ", ".join(education))
        st.write("Projects:", ", ".join(projects))
        st.write("Resume:", resume)
        st.write("---")


## Gradio UI

In [15]:
import spacy
import gradio as gr
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from faker import Faker
import random

# Load the spaCy language model
nlp = spacy.load("en_core_web_md")

# Function to extract named entities (skills, experience, education, projects, and candidate name)
def extract_entities(text):
    lines = text.split('\n')
    skills = []
    experience = []
    education = []
    projects = []
    candidate_name = None

    for line in lines:
        parts = line.strip().split(':')
        if len(parts) == 2:
            entity_type, entity_text = parts[0].strip(), parts[1].strip()
            if entity_type == "SKILL":
                skills.append(entity_text)
            elif entity_type == "EXPERIENCE":
                experience.append(entity_text)
            elif entity_type == "EDUCATION":
                education.append(entity_text)
            elif entity_type == "PROJECT":
                projects.append(entity_text)
            elif entity_type == "CANDIDATE_NAME":
                candidate_name = entity_text

    return skills, experience, education, projects, candidate_name

# Function to calculate similarity
def calculate_similarity(job_description, resume):
    all_text = [job_description, resume]
    
    # Create a CountVectorizer to convert text to a matrix of token counts
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(all_text)
    
    # Calculate cosine similarity between the job description and the resume
    cosine_similarities = cosine_similarity(X[0], X[1])

    return cosine_similarities[0][0]

# Function to generate dummy job description
def generate_dummy_job_description():
    fake = Faker()
    job_title = fake.job()
    job_description = fake.paragraphs(nb=2)
    return f"JOB_TITLE: {job_title}\nJOB_DESCRIPTION: {' '.join(job_description)}"

# Function to generate dummy resumes
def generate_dummy_resumes(num_resumes):
    fake = Faker()
    resumes = []

    for _ in range(num_resumes):
        candidate_name = fake.name()
        skills = fake.words(nb=random.randint(3, 6))
        experience = fake.paragraphs(nb=1)
        education = fake.paragraphs(nb=1)
        projects = fake.paragraphs(nb=1)
        resume = f"CANDIDATE_NAME: {candidate_name}\nSKILLS: {', '.join(skills)}\nEXPERIENCE: {' '.join(experience)}\nEDUCATION: {' '.join(education)}\nPROJECTS: {' '.join(projects)}"
        resumes.append(resume)

    return resumes

# Create Gradio interface
def resume_ranking(job_description, *resumes):
    results = []

    for resume in resumes:
        similarity_score = calculate_similarity(job_description, resume)
        skills, experience, education, projects, candidate_name = extract_entities(resume)

        results.append({
            "Candidate Name": candidate_name,
            "Similarity Score": similarity_score,
            "Skills": skills,
            "Experience": experience,
            "Education": education,
            "Projects": projects,
            "Resume": resume
        })

    # Sort results by similarity score in descending order
    results.sort(key=lambda x: x["Similarity Score"], reverse=True)
    
    # Generate an HTML table for displaying results
    html_table = "<table><tr><th>Candidate Name</th><th>Similarity Score</th><th>Skills</th><th>Experience</th><th>Education</th><th>Projects</th><th>Resume</th></tr>"
    for result in results:
        html_table += f"<tr><td>{result['Candidate Name']}</td><td>{result['Similarity Score']:.4f}</td><td>{', '.join(result['Skills'])}</td><td>{', '.join(result['Experience'])}</td><td>{', '.join(result['Education'])}</td><td>{', '.join(result['Projects'])}</td><td>{result['Resume']}</td></tr>"
    html_table += "</table>"

    return html_table

# Define Gradio interface
iface = gr.Interface(
    fn=resume_ranking,
    inputs=["text", "text", "text", "text", "text", "text", "text", "text", "text", "text"],
    outputs=gr.outputs.HTML(),
    layout="vertical",
    title="Resume Ranking App",
    description="Enter the job description and up to 9 resumes in entity format. Use the following format:\nENTITY_TYPE: ENTITY_TEXT\nExample: SKILL: Python, machine learning\nEXPERIENCE: Developed machine learning models for sentiment analysis.\nCANDIDATE_NAME: John Doe",
)

iface.launch()


C:\Users\Abhilash\AppData\Local\Temp\ipykernel_13776\2126252229.py:103: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [16]:
from IPython.display import HTML, display
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from faker import Faker
import random

# Load the spaCy language model
nlp = spacy.load("en_core_web_md")

# Function to extract named entities (skills, experience, education, projects, and candidate name)
def extract_entities(text):
    lines = text.split('\n')
    skills = []
    experience = []
    education = []
    projects = []
    candidate_name = None

    for line in lines:
        parts = line.strip().split(':')
        if len(parts) == 2:
            entity_type, entity_text = parts[0].strip(), parts[1].strip()
            if entity_type == "SKILL":
                skills.append(entity_text)
            elif entity_type == "EXPERIENCE":
                experience.append(entity_text)
            elif entity_type == "EDUCATION":
                education.append(entity_text)
            elif entity_type == "PROJECT":
                projects.append(entity_text)
            elif entity_type == "CANDIDATE_NAME":
                candidate_name = entity_text

    return skills, experience, education, projects, candidate_name

# Function to calculate similarity
def calculate_similarity(job_description, resume):
    all_text = [job_description, resume]
    
    # Create a CountVectorizer to convert text to a matrix of token counts
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(all_text)
    
    # Calculate cosine similarity between the job description and the resume
    cosine_similarities = cosine_similarity(X[0], X[1])

    return cosine_similarities[0][0]

# Function to generate dummy job description
def generate_dummy_job_description():
    fake = Faker()
    job_title = fake.job()
    job_description = fake.paragraphs(nb=2)
    return f"JOB_TITLE: {job_title}\nJOB_DESCRIPTION: {' '.join(job_description)}"

# Function to generate dummy resumes
def generate_dummy_resumes(num_resumes):
    fake = Faker()
    resumes = []

    for _ in range(num_resumes):
        candidate_name = fake.name()
        skills = fake.words(nb=random.randint(3, 6))
        experience = fake.paragraphs(nb=1)
        education = fake.paragraphs(nb=1)
        projects = fake.paragraphs(nb=1)
        resume = f"CANDIDATE_NAME: {candidate_name}\nSKILLS: {', '.join(skills)}\nEXPERIENCE: {' '.join(experience)}\nEDUCATION: {' '.join(education)}\nPROJECTS: {' '.join(projects)}"
        resumes.append(resume)

    return resumes

# Create an HTML button to generate random data
button_html = """
<button onclick="generateRandomData()">Generate Random Data</button>
"""

# JavaScript function to generate random data for job description and resumes
javascript_code = """
function generateRandomData() {
    // Generate random data for job description and resumes
    var jobDescription = generateDummyJobDescription();
    var resumes = generateDummyResumes(9);
    
    // Set the generated data in the input fields
    document.getElementById('job-description').value = jobDescription;
    for (var i = 0; i < resumes.length; i++) {
        document.getElementById('resume-' + i).value = resumes[i];
    }
}

// Function to generate dummy job description
function generateDummyJobDescription() {
    // Implement the job description generation logic here
    return "Generated Job Description";
}

// Function to generate dummy resumes
function generateDummyResumes(numResumes) {
    // Implement the resume generation logic here
    var resumes = [];
    for (var i = 0; i < numResumes; i++) {
        resumes.push("Generated Resume " + (i + 1));
    }
    return resumes;
}
"""

# Display the button and input fields
display(HTML(button_html))

# Create input fields for job description and resumes
job_description_input = gr.inputs.Textbox(label="Job Description", default=generate_dummy_job_description())
resume_inputs = [gr.inputs.Textbox(label=f"Resume {i + 1}", default=resume) for i, resume in enumerate(generate_dummy_resumes(9))]

# Create Gradio interface
iface = gr.Interface(
    fn=resume_ranking,
    inputs=[job_description_input, *resume_inputs],
    outputs=gr.outputs.HTML(),
    layout="vertical",
    title="Resume Ranking App",
    description="Enter the job description and up to 9 resumes in entity format. Use the following format:\nENTITY_TYPE: ENTITY_TEXT\nExample: SKILL: Python, machine learning\nEXPERIENCE: Developed machine learning models for sentiment analysis.\nCANDIDATE_NAME: John Doe",
)

# Launch the interface
iface.launch()


C:\Users\Abhilash\AppData\Local\Temp\ipykernel_13776\3274155016.py:113: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  job_description_input = gr.inputs.Textbox(label="Job Description", default=generate_dummy_job_description())
C:\Users\Abhilash\AppData\Local\Temp\ipykernel_13776\3274155016.py:113: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  job_description_input = gr.inputs.Textbox(label="Job Description", default=generate_dummy_job_description())
C:\Users\Abhilash\AppData\Local\Temp\ipykernel_13776\3274155016.py:113: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  job_description_input = gr.inputs.Textbox(label="Job Description", default=generate_dummy_job_description())
C:\Users\Abhilash\AppData\Local\Temp\ipykernel_13776\3274155016.py:114: GradioDeprecationWarning: Usage of gradio.inputs is depre

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# Job Descriptions:
1. We are seeking a skilled data scientist with expertise in machine learning and natural language processing (NLP) to join our team. The ideal candidate should have experience working on text classification projects.

2. We are looking for a web developer proficient in HTML, CSS, and JavaScript. The candidate should have experience building responsive and user-friendly web applications.

3. We are hiring a project manager with a strong track record of leading cross-functional teams and delivering successful projects on time and within budget.

4. We are seeking a software engineer with expertise in Python and data analysis. The ideal candidate should have experience working on big data projects.

5. We are looking for a marketing manager with experience in digital marketing, social media, and email campaigns. The candidate should have a background in marketing strategy.

6. We have an opening for a graphic designer who can create visually appealing designs for marketing materials. The candidate should be creative and detail-oriented.

7. We are seeking an experienced financial analyst to join our finance team. The candidate should have a strong background in financial modeling and forecasting.

8. We are looking for a customer support specialist to assist our customers with inquiries and issues. The candidate should have excellent communication skills.

9. We have an opening for a data entry clerk to input and manage data in our database systems. The candidate should be organized and accurate in data entry.

10. We are seeking a software quality assurance engineer to ensure the quality and reliability of our software products. The candidate should have experience in automated testing.

11. We are looking for an experienced software architect to lead the design and development of our software solutions. The candidate should have a strong background in system design and architecture.

12. We are seeking a data analyst with expertise in SQL and data visualization tools. The ideal candidate should be able to extract insights from large datasets.

13. We have an opening for a network engineer to maintain and troubleshoot our network infrastructure. The candidate should have experience with Cisco networking equipment.

14. We are looking for a content writer to create engaging and informative content for our blog and website. The candidate should have excellent writing and editing skills.

15. We are hiring a sales representative to drive revenue growth for our company. The candidate should have a proven track record in sales and client relationship management.

16. We are seeking a UX/UI designer to create user-friendly and visually appealing interfaces for our applications. The candidate should be skilled in design tools like Sketch and Figma.

17. We have an opening for a healthcare professional to join our medical team. The candidate should have a medical degree and clinical experience.

18. We are looking for a business analyst with expertise in data analysis and business process improvement. The candidate should be able to identify opportunities for optimization.

19. We are hiring a software developer with a focus on mobile app development. The candidate should have experience with iOS and Android app development.

20. We are seeking a human resources manager to oversee our HR operations. The candidate should have a strong background in HR policies and procedures.

# Resumes (One Per Line):
1. SKILL: Python, machine learning
EXPERIENCE: Developed machine learning models for sentiment analysis.
EDUCATION: Master's degree in Computer Science.
CANDIDATE_NAME: John Smith

2. SKILL: Web development, HTML, CSS, JavaScript
EXPERIENCE: Built responsive web applications for various clients.
EDUCATION: Bachelor's degree in Web Development.
CANDIDATE_NAME: Jane Doe

3. SKILL: Project management, team leadership
EXPERIENCE: Managed a team of 15+ professionals on multiple projects.
EDUCATION: MBA in Project Management.
CANDIDATE_NAME: Robert Johnson

4. SKILL: Python, data analysis, big data
EXPERIENCE: Analyzed large datasets and contributed to data-driven decisions.
EDUCATION: Bachelor's degree in Data Science.
CANDIDATE_NAME: Sarah Williams

5. SKILL: Digital marketing, social media, email campaigns
EXPERIENCE: Led digital marketing efforts, increasing online presence.
EDUCATION: Master's degree in Marketing.
CANDIDATE_NAME: Michael Davis

6. SKILL: Graphic design, Adobe Creative Suite
EXPERIENCE: Created visually appealing marketing materials for clients.
EDUCATION: Bachelor's degree in Graphic Design.
CANDIDATE_NAME: Emily Brown

7. SKILL: Financial modeling, forecasting
EXPERIENCE: Prepared financial reports and forecasts for a Fortune 500 company.
EDUCATION: Master's degree in Finance.
CANDIDATE_NAME: Christopher Lee

8. SKILL: Customer support, communication
EXPERIENCE: Assisted customers with inquiries and resolved issues promptly.
EDUCATION: Bachelor's degree in Business Administration.
CANDIDATE_NAME: Amanda Wilson

9. SKILL: Data entry, database management
EXPERIENCE: Maintained accurate records in database systems.
EDUCATION: High school diploma.
CANDIDATE_NAME: Daniel Garcia

10. SKILL: Quality assurance, automated testing
EXPERIENCE: Ensured software quality through rigorous testing procedures.
EDUCATION: Bachelor's degree in Computer Science.
CANDIDATE_NAME: Olivia Martinez

11. SKILL: Software architecture, system design
EXPERIENCE: Led the design and development of complex software systems.
EDUCATION: Master's degree in Computer Science.
CANDIDATE_NAME: David Miller

12. SKILL: Data analysis, SQL, data visualization
EXPERIENCE: Extracted insights from large datasets to drive business decisions.
EDUCATION: Bachelor's degree in Data Science.
CANDIDATE_NAME: Jennifer Brown

13. SKILL: Network maintenance, troubleshooting, Cisco equipment
EXPERIENCE: Managed and maintained a large network infrastructure.
EDUCATION: Cisco Certified Network Professional (CCNP).
CANDIDATE_NAME: Michael Johnson

14. SKILL: Content writing, editing
EXPERIENCE: Created engaging content for blogs and websites.
EDUCATION: Bachelor's degree in English.
CANDIDATE_NAME: Sarah Anderson

15. SKILL: Sales, client relationship management
EXPERIENCE: Achieved consistent revenue growth through effective sales strategies.
EDUCATION: Bachelor's degree in Business Administration.
CANDIDATE_NAME: John Davis

16. SKILL: UX/UI design, Sketch, Figma
EXPERIENCE: Designed user-friendly interfaces for web and mobile applications.
EDUCATION: Master's degree in Interaction Design.
CANDIDATE_NAME: Emily Smith

17. SKILL: Medical degree, clinical experience
EXPERIENCE: Practiced medicine in various clinical settings.
EDUCATION: Doctor of Medicine (MD).
CANDIDATE_NAME: Dr. Robert Anderson

18. SKILL: Business analysis, data analysis, process improvement
EXPERIENCE: Identified process optimization opportunities, leading to cost savings.
EDUCATION: MBA in Business Analytics.
CANDIDATE_NAME: Jessica Wilson

19. SKILL: Mobile app development, iOS, Android
EXPERIENCE: Developed and launched mobile apps on iOS and Android platforms.
EDUCATION: Bachelor's degree in Computer Engineering.
CANDIDATE_NAME: Andrew Martinez

20. SKILL: Human resources management, HR policies
EXPERIENCE: Oversaw HR operations for a large organization.
EDUCATION: Master's degree in Human Resources.
CANDIDATE_NAME: Megan Johnson
